In [1]:
import json
import yaml
import pandas as pd
import numpy as np

from tqdm import tqdm

In [2]:
businesses = []

In [3]:
with open('RESERVE/yelp_dataset/yelp_academic_dataset_business.json', 'r') as b:
    for business in b:
        business = json.loads(business)
        bus_id, categories = business['business_id'], business['categories']
        attributes = business.get("attributes", None)

        if categories:
            business_to_add = {'business_id': bus_id, 'categories': categories}
            if (attributes) and (attributes.get("Ambience", None) is not None): business_to_add['ambience'] = attributes["Ambience"]
            businesses.append(business_to_add)

In [4]:
len('bvN78flM8NLprQ1a1y5dRg')

22

In [5]:
len(businesses)

160470

In [6]:
restaurants = []
others = []

In [7]:
for business in businesses:
    if 'Restaurants' in business['categories']:
        restaurants.append(business)
    else:
        others.append(business)

In [20]:
for business in others:
    business['categories'] = ['Non_restaurant']

In [21]:
ambient_data = []

In [22]:
for business in tqdm(businesses):
    try:
        if 'ambience' in business:
            ambient_row = {'business_id': business['business_id']}
            ambient_row.update(yaml.load(business['ambience'], Loader=yaml.FullLoader))
            ambient_data.append(ambient_row)
    except ValueError:
        pass

100%|██████████| 160470/160470 [00:26<00:00, 5954.54it/s]


In [23]:
ambient_df = pd.DataFrame(ambient_data)

In [24]:
ambient_df.head()

,business_id,touristy,hipster,romantic,divey,intimate,trendy,upscale,classy,casual
0,6iYb2HFDywm3zjuRg0shjw,False,False,False,False,False,False,False,False,True
1,tCbdrRPZA0oiIYSmHG3J0w,False,False,False,False,False,False,False,False,True
2,D4JtQNTI4X3KcbzacDJsMw,False,False,False,False,False,False,False,False,True
3,HPA_qyMEddpAEtFof02ixg,False,False,False,False,False,False,False,False,True
4,dmbbf3AqeG61_QHRZi1M1w,False,False,False,False,False,False,False,False,False


In [25]:
ambient_df.to_csv('yelp_academic/data/business_ambience.csv', index = False)

### Ambience

In [57]:
photos_ambience = []

In [59]:
with open('yelp_photos/photos.json', 'r') as p:
    for photo in p:
        photo = json.loads(photo)
        photos_ambience.append({'photo_id': photo['photo_id'],
                      'business_id': photo['business_id']})

In [64]:
photos_ambience = pd.DataFrame(photos_ambience)

In [65]:
photos_ambience.head()

,photo_id,business_id
0,Un_Og6jfhazVn7CxszkKEw,R1sIqGfKpF5f3HV3vyNsbg
1,BFE1AFOs27scnnfeBf99ZA,vdT7zlrLB2DL9pStDUs91A
2,7t-C0r1JRdoVD9FS7M-N7Q,c5GzrObEdj7eNVOdAxrppg
3,rLnw0d-YYZvT9kR4y7h7_Q,aQa7N5ZbPhCoKYGGB-gqfg
4,Cv5M8MDw8a5NEWvw2AQ4nw,B-s6qOFD75syhwWjp518aA


In [67]:
photos_ambience = pd.merge(photos_ambience, ambient_df, on='business_id')

In [68]:
photos_ambience = photos_ambience[photos_ambience['photo_id'].notna()]

In [69]:
photos_ambience = photos_ambience[photos_ambience['business_id'].notna()]

In [79]:
obj_columns = list(photos_ambience.select_dtypes(include=['object']).columns.values)
photos_ambience[obj_columns] = photos_ambience[obj_columns].replace([None], np.nan).replace('None', np.nan)

In [81]:
photos_ambience = photos_ambience.fillna(False)

In [84]:
photos_ambience = photos_ambience[photos_ambience[['touristy', 'hipster', 'romantic', 'divey', 'intimate', 'trendy', 'upscale', 'classy', 'casual']].any(axis=1)]

In [85]:
amb_train, amb_dev = train_test_split(photos_ambience, test_size = 0.2, random_state = 42)
amb_dev, amb_test = train_test_split(amb_dev, test_size = 0.5, random_state = 42)

In [87]:
def df_ambience(df):
    amb_counts = {'touristy': 0,
                  'hipster': 0,
                  'romantic': 0,
                  'divey': 0,
                  'intimate': 0,
                  'trendy': 0,
                  'upscale': 0,
                  'classy': 0,
                  'casual': 0}
    for amb in amb_counts:
        amb_counts[amb] = len(df.loc[df[amb]  == True])
    return len(df), amb_counts

In [91]:
df_ambience(amb_test)

(15074,
 {'touristy': 724,
  'hipster': 1293,
  'romantic': 463,
  'divey': 318,
  'intimate': 409,
  'trendy': 3135,
  'upscale': 620,
  'classy': 7342,
  'casual': 11363})

In [159]:
amb_train = amb_train.reset_index(drop=True)
amb_dev = amb_dev.reset_index(drop=True)
amb_test = amb_test.reset_index(drop=True)

In [160]:
amb_train.to_csv('yelp_data/business_ambience_train.csv', index = False)
amb_dev.to_csv('yelp_data/business_ambience_dev.csv', index = False)
amb_test.to_csv('yelp_data/business_ambience_test.csv', index = False)

### Restaurant or not

In [131]:
for business in businesses:
    if business['categories'] == ['Non_restaurant']:
        business['restaurant'] = False
    else:
        business['restaurant'] = True
    del business['categories']

In [134]:
for business in businesses:
    if 'ambience' in business:
        del business['ambience']

In [140]:
businesses = pd.DataFrame(businesses)

In [137]:
photos_businesses_all = []

In [138]:
with open('yelp_photos/photos.json', 'r') as p:
    for photo in p:
        photo = json.loads(photo)
        photos_businesses_all.append({'photo_id': photo['photo_id'],
                      'business_id': photo['business_id']})

In [139]:
rest_or_not = pd.DataFrame(photos_businesses_all)

In [141]:
rest_or_not = pd.merge(rest_or_not, businesses, on='business_id')

In [144]:
rest_or_not.head()

,photo_id,business_id,restaurant
0,Un_Og6jfhazVn7CxszkKEw,R1sIqGfKpF5f3HV3vyNsbg,True
1,vjjyXgKxc1Wu6v0aaHLFuQ,R1sIqGfKpF5f3HV3vyNsbg,True
2,VWWdSBLn7f2eaY16GruQOg,R1sIqGfKpF5f3HV3vyNsbg,True
3,fSJ2--legv4crDHIJWIOmA,R1sIqGfKpF5f3HV3vyNsbg,True
4,liRGQZfixy5anDcjCqGuPA,R1sIqGfKpF5f3HV3vyNsbg,True


In [145]:
bus_train, bus_dev = train_test_split(rest_or_not, test_size = 0.2, random_state = 42)
bus_dev, bus_test = train_test_split(bus_dev, test_size = 0.5, random_state = 42)

In [163]:
bus_train

,photo_id,business_id,restaurant
0,WGp63RIuT6C4qxboG3FgpQ,J7Hs5_PGlZajNRxGptGRvw,True
1,eKa8I6RGgi7W_HOpqFOgtA,LnBBOksOrzUiiDQkjUeMdg,True
2,jx9rraUeZ3JvazMvFLw0yw,MkGPncZD8oqp-AHkLW740g,True
3,8E3MwgLc-ggdDfokoYPd9Q,Ur7o-5FviIC9YmJqsGKR6A,True
4,eLVN8Y2DSSU2Gt4haHxwzA,KbyrRWuEeK6uEbtpAUgIdg,True
...,...,...,...
159995,aMvsc1kffaZU-c2qCRibTg,vBKwHwBqkFdxw4VetbYSxQ,True
159996,Fl4je3JGtt2h_UwrKo3xMw,3u3Xllz8NezyMjH6zsGp9w,True
159997,mErOCowqwADIKpLf98Rgmg,Xge6ajeuR7CEio9vSdFsJA,True
159998,TjjzxVtqKbSwT0NGFERbPA,8tLzmoG8Dluy80_iULO-Xg,True


In [151]:
len(bus_dev)

20000

In [161]:
bus_train = bus_train.reset_index(drop=True)
bus_dev = bus_dev.reset_index(drop=True)
bus_test = bus_test.reset_index(drop=True)

In [162]:
bus_train.to_csv('yelp_data/business_restaurant_train.csv', index = False)
bus_dev.to_csv('yelp_data/business_restaurant_dev.csv', index = False)
bus_test.to_csv('yelp_data/business_restaurant_test.csv', index = False)